## Regression Model In Keras - Final Project - Part A

In this project I'll build a regression model using the deep learing Keras library, and then experiment with increasing the number of training epochs and changing number of hidden layers and see how changing these parameters impact the performance of the model.

This project is the conclusion of the course 'Introduction to Deep Learning and Neural Networds with Keras' by IBM\Coursera. This is the first part out of 4 where I'll show the diferences reporting the mean and the Standard Deviation of the Mean Squared Error.

### Concrete Dataset

The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Igredientes include:

1. Cement

2. Blast Furnace Slag

3. Fly Ash

4. Water

5. Superplasticizer

6. Coarse Aggregate

7. Fine Aggregate

In [1]:
import pandas as pd
import numpy as np
import keras

In [2]:
# Downloand the data
# is the same dataset using in the course

concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa(megapascal).

In [3]:
# checking data points

concrete_data.shape

(1030, 9)

There are approximately 1000 samples to train the model on.

In [4]:
# check for any missing values

concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The datasete looks clean and ready to be used to built a model.

### Split data into predictors and target

The dependent variable in this problem is the conrete sample strength. Therefore, the independent variable will be the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [7]:
# check the predictors column
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
# check the target column
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

The last step is normalize the data by substracting the mean and dividing by the standard deviation.

In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [10]:
# saving the num of predictors to n_cols since we'll need theis num when building the network

n_cols = predictors_norm.shape[1] # num of predictors
n_cols

8

In [11]:
# step 1
# split randomly into train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size = 0.30, random_state = 42)

print("number of test samples :", X_test.shape[0])
print("number of training samples:",X_train.shape[0])

number of test samples : 309
number of training samples: 721


### Build a neural network

Define a function that defines the regression model so that can conveniently call it to create the model

In [12]:
# importing the rest ov the packages from Keras library that will need for the regression model

from keras.models import Sequential
from keras.layers import Dense

In [13]:
# define regression model

def regression_model():
    # create model
    model = Sequential()
    # two hidden layers and 50 nodes and ReLu activation function
    model.add(Dense(50, activation = 'relu', input_shape = (n_cols,)))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(1))
    
    # compile model
    # use 'adam' optimizer and mse as loss function
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model

### Train and Test the Network

Call the function now to create the model

In [14]:
# build the model
model = regression_model()

In [15]:
# step 2
# train the model on train data using 50 epochs

model.fit(X_train, y_train, validation_split = 0.3, epochs = 50, verbose = 2)

Epoch 1/50
23/23 - 1s - loss: 5428.4863 - 1s/epoch - 44ms/step
Epoch 2/50
23/23 - 0s - loss: 800.3329 - 80ms/epoch - 3ms/step
Epoch 3/50
23/23 - 0s - loss: 293.8068 - 54ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 206.5061 - 55ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 177.6862 - 55ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 158.2678 - 60ms/epoch - 3ms/step
Epoch 7/50
23/23 - 0s - loss: 139.1327 - 56ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 126.9861 - 56ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 117.9180 - 53ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 104.7566 - 55ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 94.5578 - 54ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 87.9543 - 60ms/epoch - 3ms/step
Epoch 13/50
23/23 - 0s - loss: 80.0912 - 54ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 84.9585 - 60ms/epoch - 3ms/step
Epoch 15/50
23/23 - 0s - loss: 75.5475 - 54ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 80.3629 - 56ms/epoch - 

In [31]:
# step 3
# evaluate the model 
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

10/10 [==============================] - 0s 3ms/step


93.23717498779297

In [35]:
# importing mean squared error
from sklearn.metrics import mean_squared_error

mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
mean

93.23718163294144

In [36]:
stdv = np.std(mean_square_error)
stdv

0.0

In [37]:
# step 4
# repeat step 1 - 3, 50 times(creat a list of 50 mse)
import tensorflow

total_mean_squared_errors = 50
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size = 0.3, random_state = 42)
    model.fit(X_train, y_train, epochs=50, verbose = 2)
    MSE = model.evaluate(X_test, y_test, verbose = 2)
    print('MSE '+str(i + 1)+': '+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)
    
mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

Epoch 1/50
23/23 - 0s - loss: 3222557.7500 - 53ms/epoch - 2ms/step
Epoch 2/50
23/23 - 0s - loss: 59921.4688 - 65ms/epoch - 3ms/step
Epoch 3/50
23/23 - 0s - loss: 5374.1890 - 59ms/epoch - 3ms/step
Epoch 4/50
23/23 - 0s - loss: 1095.6296 - 61ms/epoch - 3ms/step
Epoch 5/50
23/23 - 0s - loss: 970.7997 - 56ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 886.2729 - 62ms/epoch - 3ms/step
Epoch 7/50
23/23 - 0s - loss: 831.1196 - 60ms/epoch - 3ms/step
Epoch 8/50
23/23 - 0s - loss: 785.1337 - 55ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 749.0721 - 57ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 719.7509 - 56ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 694.2259 - 57ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 672.2859 - 55ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 651.4507 - 56ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 632.6317 - 55ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 615.6874 - 62ms/epoch - 3ms/step
Epoch 16/50
23/23 - 0s - loss: 599.3279 -

Epoch 26/50
23/23 - 0s - loss: 117.1868 - 55ms/epoch - 2ms/step
Epoch 27/50
23/23 - 0s - loss: 116.1227 - 55ms/epoch - 2ms/step
Epoch 28/50
23/23 - 0s - loss: 115.4628 - 54ms/epoch - 2ms/step
Epoch 29/50
23/23 - 0s - loss: 115.3352 - 53ms/epoch - 2ms/step
Epoch 30/50
23/23 - 0s - loss: 114.8389 - 56ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 114.3949 - 53ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 114.2591 - 53ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 113.9364 - 57ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 113.5487 - 53ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 113.0112 - 56ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 113.7323 - 92ms/epoch - 4ms/step
Epoch 37/50
23/23 - 0s - loss: 112.6224 - 62ms/epoch - 3ms/step
Epoch 38/50
23/23 - 0s - loss: 112.1327 - 53ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 112.1562 - 54ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 111.8750 - 52ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 111.6543 

MSE 5: 81.2280044555664
10/10 [==============================] - 0s 3ms/step
Epoch 1/50
23/23 - 0s - loss: 74.0533 - 66ms/epoch - 3ms/step
Epoch 2/50
23/23 - 0s - loss: 77.6896 - 67ms/epoch - 3ms/step
Epoch 3/50
23/23 - 0s - loss: 74.7169 - 66ms/epoch - 3ms/step
Epoch 4/50
23/23 - 0s - loss: 74.6775 - 64ms/epoch - 3ms/step
Epoch 5/50
23/23 - 0s - loss: 74.6319 - 60ms/epoch - 3ms/step
Epoch 6/50
23/23 - 0s - loss: 77.4655 - 59ms/epoch - 3ms/step
Epoch 7/50
23/23 - 0s - loss: 73.9584 - 52ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 72.3457 - 53ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 71.9733 - 57ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 71.5465 - 56ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 72.0926 - 53ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 70.7519 - 57ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 71.0284 - 53ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 70.9152 - 57ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 70.3974 - 69ms/epoc

23/23 - 0s - loss: 49.6776 - 64ms/epoch - 3ms/step
Epoch 27/50
23/23 - 0s - loss: 49.6879 - 64ms/epoch - 3ms/step
Epoch 28/50
23/23 - 0s - loss: 53.6147 - 63ms/epoch - 3ms/step
Epoch 29/50
23/23 - 0s - loss: 48.7105 - 62ms/epoch - 3ms/step
Epoch 30/50
23/23 - 0s - loss: 49.7093 - 62ms/epoch - 3ms/step
Epoch 31/50
23/23 - 0s - loss: 47.9406 - 62ms/epoch - 3ms/step
Epoch 32/50
23/23 - 0s - loss: 52.7998 - 62ms/epoch - 3ms/step
Epoch 33/50
23/23 - 0s - loss: 54.3878 - 62ms/epoch - 3ms/step
Epoch 34/50
23/23 - 0s - loss: 47.6151 - 62ms/epoch - 3ms/step
Epoch 35/50
23/23 - 0s - loss: 49.8999 - 95ms/epoch - 4ms/step
Epoch 36/50
23/23 - 0s - loss: 50.4721 - 74ms/epoch - 3ms/step
Epoch 37/50
23/23 - 0s - loss: 52.3345 - 64ms/epoch - 3ms/step
Epoch 38/50
23/23 - 0s - loss: 48.7521 - 64ms/epoch - 3ms/step
Epoch 39/50
23/23 - 0s - loss: 47.7538 - 67ms/epoch - 3ms/step
Epoch 40/50
23/23 - 0s - loss: 48.0933 - 62ms/epoch - 3ms/step
Epoch 41/50
23/23 - 0s - loss: 47.9629 - 62ms/epoch - 3ms/step
Epoc

10/10 [==============================] - 0s 3ms/step
Epoch 1/50
23/23 - 0s - loss: 50.9538 - 60ms/epoch - 3ms/step
Epoch 2/50
23/23 - 0s - loss: 41.0713 - 58ms/epoch - 3ms/step
Epoch 3/50
23/23 - 0s - loss: 41.2678 - 57ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 41.6791 - 57ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 40.7816 - 56ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 46.0296 - 53ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 44.7609 - 54ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 43.6883 - 57ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 44.9614 - 58ms/epoch - 3ms/step
Epoch 10/50
23/23 - 0s - loss: 43.4285 - 55ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 43.1674 - 56ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 39.8424 - 69ms/epoch - 3ms/step
Epoch 13/50
23/23 - 0s - loss: 42.8702 - 71ms/epoch - 3ms/step
Epoch 14/50
23/23 - 0s - loss: 43.6533 - 69ms/epoch - 3ms/step
Epoch 15/50
23/23 - 0s - loss: 44.2157 - 71ms/epoch - 3ms/step
Epoch 16/50

...

Epoch 31/50
23/23 - 0s - loss: 15.1427 - 58ms/epoch - 3ms/step
Epoch 32/50
23/23 - 0s - loss: 13.9747 - 53ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 15.7840 - 53ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 16.8118 - 52ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 16.0600 - 56ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 14.7508 - 52ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 14.6276 - 57ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 17.3133 - 53ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 15.6092 - 60ms/epoch - 3ms/step
Epoch 40/50
23/23 - 0s - loss: 16.2560 - 57ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 15.3296 - 53ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 14.9278 - 53ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 14.3056 - 53ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 14.7165 - 57ms/epoch - 2ms/step
Epoch 45/50
23/23 - 0s - loss: 14.6742 - 52ms/epoch - 2ms/step
Epoch 46/50
23/23 - 0s - loss: 15.8170 - 56ms/epoch - 2

23/23 - 0s - loss: 14.8267 - 53ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 16.4616 - 54ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 15.0821 - 52ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 14.5593 - 58ms/epoch - 3ms/step
Epoch 9/50
23/23 - 0s - loss: 13.2640 - 55ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 14.9262 - 54ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 14.6159 - 53ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 14.3963 - 57ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 13.8135 - 54ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 14.0126 - 57ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 14.2475 - 54ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 13.9580 - 56ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 14.9825 - 53ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 13.8006 - 59ms/epoch - 3ms/step
Epoch 19/50
23/23 - 0s - loss: 13.9469 - 57ms/epoch - 2ms/step
Epoch 20/50
23/23 - 0s - loss: 15.6422 - 53ms/epoch - 2ms/step
Epoch 21

Epoch 32/50
23/23 - 0s - loss: 14.0322 - 77ms/epoch - 3ms/step
Epoch 33/50
23/23 - 0s - loss: 13.0254 - 76ms/epoch - 3ms/step
Epoch 34/50
23/23 - 0s - loss: 13.7628 - 75ms/epoch - 3ms/step
Epoch 35/50
23/23 - 0s - loss: 13.1414 - 73ms/epoch - 3ms/step
Epoch 36/50
23/23 - 0s - loss: 13.8217 - 59ms/epoch - 3ms/step
Epoch 37/50
23/23 - 0s - loss: 13.3297 - 54ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 13.8100 - 53ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 12.5959 - 53ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 13.9773 - 56ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 14.4177 - 53ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 13.6546 - 57ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 14.2890 - 54ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 14.8318 - 59ms/epoch - 3ms/step
Epoch 45/50
23/23 - 0s - loss: 18.5252 - 76ms/epoch - 3ms/step
Epoch 46/50
23/23 - 0s - loss: 14.0407 - 76ms/epoch - 3ms/step
Epoch 47/50
23/23 - 0s - loss: 13.3803 - 76ms/epoch - 3

Epoch 7/50
23/23 - 0s - loss: 13.6462 - 83ms/epoch - 4ms/step
Epoch 8/50
23/23 - 0s - loss: 13.6317 - 67ms/epoch - 3ms/step
Epoch 9/50
23/23 - 0s - loss: 14.1189 - 62ms/epoch - 3ms/step
Epoch 10/50
23/23 - 0s - loss: 14.3909 - 61ms/epoch - 3ms/step
Epoch 11/50
23/23 - 0s - loss: 13.9951 - 68ms/epoch - 3ms/step
Epoch 12/50
23/23 - 0s - loss: 15.1404 - 64ms/epoch - 3ms/step
Epoch 13/50
23/23 - 0s - loss: 18.7901 - 63ms/epoch - 3ms/step
Epoch 14/50
23/23 - 0s - loss: 15.9059 - 60ms/epoch - 3ms/step
Epoch 15/50
23/23 - 0s - loss: 13.4655 - 63ms/epoch - 3ms/step
Epoch 16/50
23/23 - 0s - loss: 13.3934 - 65ms/epoch - 3ms/step
Epoch 17/50
23/23 - 0s - loss: 13.5044 - 64ms/epoch - 3ms/step
Epoch 18/50
23/23 - 0s - loss: 13.8613 - 55ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 13.4911 - 58ms/epoch - 3ms/step
Epoch 20/50
23/23 - 0s - loss: 13.7000 - 57ms/epoch - 2ms/step
Epoch 21/50
23/23 - 0s - loss: 13.9331 - 53ms/epoch - 2ms/step
Epoch 22/50
23/23 - 0s - loss: 13.9366 - 54ms/epoch - 2ms/

Epoch 34/50
23/23 - 0s - loss: 12.8874 - 58ms/epoch - 3ms/step
Epoch 35/50
23/23 - 0s - loss: 13.4443 - 59ms/epoch - 3ms/step
Epoch 36/50
23/23 - 0s - loss: 13.5237 - 55ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 12.1716 - 63ms/epoch - 3ms/step
Epoch 38/50
23/23 - 0s - loss: 13.4936 - 65ms/epoch - 3ms/step
Epoch 39/50
23/23 - 0s - loss: 12.7066 - 64ms/epoch - 3ms/step
Epoch 40/50
23/23 - 0s - loss: 12.8051 - 64ms/epoch - 3ms/step
Epoch 41/50
23/23 - 0s - loss: 14.0810 - 60ms/epoch - 3ms/step
Epoch 42/50
23/23 - 0s - loss: 12.5660 - 55ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 15.1103 - 107ms/epoch - 5ms/step
Epoch 44/50
23/23 - 0s - loss: 13.5169 - 59ms/epoch - 3ms/step
Epoch 45/50
23/23 - 0s - loss: 14.0436 - 60ms/epoch - 3ms/step
Epoch 46/50
23/23 - 0s - loss: 14.3744 - 59ms/epoch - 3ms/step
Epoch 47/50
23/23 - 0s - loss: 13.7118 - 59ms/epoch - 3ms/step
Epoch 48/50
23/23 - 0s - loss: 12.1792 - 57ms/epoch - 2ms/step
Epoch 49/50
23/23 - 0s - loss: 12.6417 - 54ms/epoch - 

In [38]:
# print mse and standard deviation

print('Mean: '+ str(mean))
print('Standard Deviation: '+str(standard_deviation))

Mean: 47.42956444590453
Standard Deviation: 40.345654060755
